In [81]:
# ! pip install crewai crewai_tools openai langchain


In [82]:
import os
import sys
try:
    get_ipython
    current_dir = os.getcwd()
except NameError:
    current_dir = os.path.dirname(os.path.abspath(__file__))

# Set path，temporary path expansion
project_dir = os.path.abspath(os.path.join(current_dir, '..'))
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from dotenv import load_dotenv  # 导入python-dotenv

from crewai import Agent, Task, Crew, Process,LLM
from crewai_tools import (
    SerperDevTool,       # 网络搜索
    CSVSearchTool,      # 分析CSV数据
    DirectoryReadTool,
    FileReadTool,
)
from crewai.tools import BaseTool,tool
from openai import OpenAI
from pydantic import Field



### Load Config


In [83]:
load_dotenv()
BASE_URL= "https://dashscope.aliyuncs.com/compatible-mode/v1"
ONLINE_MODEL_NAME ="qwen-plus"
api_key = (
            os.getenv("QWEN_API_KEY")
            or os.getenv("DASHSCOPE_API_KEY")
        ) 


In [84]:
product_name ="Samsung Z Flip7"

### Define LLM

In [85]:

llm = LLM(
            base_url=BASE_URL,
            api_key=api_key,
            model="openai/"+ONLINE_MODEL_NAME,  # 本次使用的模型
            # temperature=0.7,  # 发散的程度
            # timeout=None,
            # max_retries=2,
        )




### init tools

In [ ]:


@tool("竞品分析",result_as_answer=True)    
def compare_products(target_product: str, competitors: list, metrics: list) -> str:
    """
    对比目标产品与竞品的关键指标
    
    :param target_product: 目标产品名称
    :param competitors: 竞品列表
    :param metrics: 要对比的指标列表（如价格、市场份额、用户评分等）
    :return: 对比分析结果字符串
    """
    # 模拟生成对比数据（实际应用中可对接真实数据源）
    result = f"产品对比分析（目标产品：{target_product}）\n"
    result += "".join([f"- 对比竞品：{', '.join(competitors)}\n",
                      f"- 分析指标：{', '.join(metrics)}\n\n"])
    
    # 生成模拟数据
    for metric in metrics:
        # 为每个指标生成随机数据用于演示
        target_value = round(5 + (hash(target_product + metric) % 5), 1)
        result += f"{metric}：\n"
        result += f"  - {target_product}：{target_value}\n"
        
        for competitor in competitors:
            comp_value = round(target_value + (hash(competitor + metric) % 3 - 1), 1)
            result += f"  - {competitor}：{comp_value}（{('高于' if comp_value > target_value else '低于') if comp_value != target_value else '持平'}）\n"
        result += "\n"
        
    return result + "数据来源：竞品分析工具模拟数据"



In [87]:
docs_tool = DirectoryReadTool(directory='../results')
file_tool = FileReadTool()
search_tool = SerperDevTool()
# csv_tool=CSVSearchTool()
# web_rag_tool = WebsiteSearchTool()


### Define Agents

In [88]:
# 创建市场研究员智能体
market_researcher = Agent(
    role="市场研究员",
    goal="收集并分析目标产品的市场数据和竞争格局",
    backstory="拥有5年市场研究经验，擅长分析产品定位和竞争优势，"
              "能够从复杂数据中提取有价值的市场洞察。",
    tools=[search_tool],
    llm=llm,
    system_prompt_suffix="""
    输出规则：
    1. 所有数据必须注明来源（如"根据XX工具查询结果"）
    2. 若数据不完整，明确说明缺失部分，不进行猜测
    3. 分析结论必须包含数据支持，禁止主观判断
    """,
    verbose=True
)

# 创建数据分析师智能体
data_analyst = Agent(
    role="数据分析师",
    goal="将市场数据可视化并提供数据支持",
    backstory="数据科学专家，擅长将市场数据转化为直观的图表和数据摘要，"
              "帮助团队理解数据背后的含义。",
    tools=[file_tool,compare_products],
    llm=llm,
    verbose=True
)

# 创建内容创作者智能体
content_creator = Agent(
    role="内容创作者",
    goal="基于市场洞察创作吸引人的产品推广文案",
    backstory="资深文案策划，擅长将复杂的产品优势转化为简洁有力的推广语言，"
              "熟悉社交媒体和广告文案风格。",
    tools=[docs_tool,file_tool],
    llm=llm,
    verbose=True
)

### Define Task

In [89]:
# 定义任务
datapath ='./results/datas.md' 
task1 = Task(
    description=f"收集并分析'{product_name}'的市场数据，包括竞争对手、用户评价和市场趋势",
    expected_output="一份详细的市场分析报告，包含关键洞察和机会点",
    agent=market_researcher,
    create_directory=True,
    output_file= datapath
)

task2 = Task(
    description=f"读取{datapath}内容，总结市场分析的关键数据点，生成数据摘要和竞品分析",
    expected_output="一份包含核心数据和关键发现的数据摘要,一份竞品分析的报告",
    agent=data_analyst,
    context=[task1] , # 依赖任务1的结果
  
)


task3 = Task(
    description=f"基于市场分析报告，为'{product_name}'创作3条适合社交媒体的推广文案，"
                "突出产品优势和差异化特点",
    expected_output="3条简洁有力、符合社交媒体风格的推广文案",
    agent=content_creator,
    context=[task1,task2],  
    output_file= os.path.join(project_dir,'results/new_post.md' )
)



### Define Crew

In [90]:
# 创建智能体团队
crew = Crew(
    agents=[market_researcher, content_creator, data_analyst],
    tasks=[ task2, task3],
    process=Process.sequential  # 按顺序执行任务
)



### Start Agent

In [91]:
# 启动团队协作
results = crew.kickoff()
results

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 数据分析师                                                                                              │
│                                                                                                                 │
│  Task: 读取./results/datas.md内容，总结市场分析的关键数据点，生成数据摘要和竞品分析                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 数据分析师                                                                                              │
│                                                                                                                 │
│  Thought: 我需要读取文件内容以获取市场分析的关键数据点。                                                        │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"file_path\": \"./results/datas.md\"}"                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  **Samsung Galaxy Z Flip7 市场分析报告**                                                                        │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **1. 产品概述**                                                                                            │
│  **发布日期**：Samsung Galaxy Z Flip7 于2025年7月9日正式发布，并于7月25日正式上市。                             │
│  **市场定位**：Z Flip7 是三星折叠屏系列的最新成员，主打紧凑型折叠设计，适合时尚潮流引领者和科技爱好者。         │
│  **核心功能**：                                                                                                 │
│  - 更大的外部AMOLED显示屏（120Hz刷新率）                                                                        │
│  - 50 MP自拍相机                                                                                                │
│  - 改进的铰链设计，支持更长的耐用性                                                                             │
│  - 更轻薄的机身设计                                                                                             │
│  - Galaxy AI功能增强                                                                                            │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **2. 竞争对手分析**                                                                                        │
│  #### **主要竞争对手**                                                                                          │
│  1. **Motorola Razr (2025)** 和 **Razr Ultra**                                                                  │
│     - 优势：更亲民的价格、更成熟的折叠设计                                                                      │
│     - 劣势：性能略逊于Z Flip7，软件体验较弱                                                                     │
│                                                                                                                 │
│  2. **Samsung Galaxy Z Fold7**                                                                                  │
│     - 作为三星自家产品，Z Fold7主打大屏体验，而Z Flip7则主打便携性，两者形成互补关系。                          │
│                                                                                                                 │
│  3. **OnePlus 13** 和 **Samsung Galaxy S25 Plus**                                                               │
│     - 优势：旗舰级性能、更成熟的生态系统                                                                        │
│     - 劣势：缺乏创新的折叠设计，吸引力较低                                                                      │
│                                                                                                                 │
│  4. **iPhone 14 Pro Max**（高端市场）                                                                           │
│     - 优势：iOS生态系统、品牌忠诚度高                                                                           │
│     - 劣势：缺乏创新设计，价格较高                                                                              │
│                                                                                                                 │
│  #### **市场格局**                                                                                              │
│  - 折叠屏市场增长显著，2024年全球出货量达1720万台，同比增长12%。                                                │
│  - 三星在折叠屏市场的

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 数据分析师                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  **数据摘要**                                                                                                   │
│                                                                                                                 │
│  1. **产品概述**                                                                                                │
│     - **发布日期**：2025年7月9日（正式发布），7月25日上市                                                       │
│     - **市场定位**：紧凑型折叠屏手机，面向时尚潮流引领者和科技爱好者                                            │
│     - **核心功能**：                                                                                            │
│       - 外部AMOLED显示屏（120Hz刷新率）                                                                         │
│       - 50 MP自拍相机                                                                                           │
│       - 改进的铰链设计（500,000次折叠寿命）                                                                     │
│       - 更轻薄机身设计                                                                                          │
│       - Galaxy AI功能增强                                                                                       │
│                                                                                                                 │
│  2. **市场趋势**                                                                                                │
│     - 折叠屏市场增长显著，2024年全球出货量达1720万台，同比增长12%                                               │
│     - 三星在折叠屏市场的份额从54%下降至45%                                                                      │
│     - 预计到2026年，折叠屏将占三星美国市场销量的15-20%                                                          │
│                                                                                                                 │
│  3. **用户反馈**                                                                                                │
│     - **正面评价**：外观设计时尚、外部屏幕便捷、相机表现优异、性能流畅                                          │
│     - **负面评价**：价格昂贵（起售价£1,049）、耐用性担忧、电池续航不足、缺少长焦镜头                            │
│                                                                                                                 │
│  4. **关键洞察**                                                                                                │
│     - Z Flip7是三星折叠屏系列的亮点产品，成功吸引年轻用户和潮流引领者                                           │
│     - 折叠屏市场竞争加剧，尤其是来自Motorola、Google和中国品牌的挑战                                            │
│     - 用户对折叠屏接受度逐步提高，但价格和耐用性仍是主要顾虑                                                    │
│     - AI技术的集成为Z Flip7带来了差异化优势                                                                     │
│                                                                                                                 │
│  5. **机会点**                                                                                                  │
│     - 优化电池续航                                                                                              │
│     - 推出更多价格层级（如Z Flip7 FE）                                                                          │
│     - 加强耐用性宣传                                                                                            │
│     - 拓展AI应用场景（如AI摄影、语音助手、健康监测）                                   

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 内容创作者                                                                                              │
│                                                                                                                 │
│  Task: 基于市场分析报告，为'Samsung Z Flip7'创作3条适合社交媒体的推广文案，突出产品优势和差异化特点             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 内容创作者                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. 📱✨ 折叠未来，掌心绽放！Samsung Galaxy Z                                                                   │
│  Flip7，以500,000次耐用铰链与轻薄设计，重新定义潮流手机。外屏AMOLED 120Hz刷新率，一瞥即惊艳。#ZFlip7            │
│  #折叠屏新纪元                                                                                                  │
│                                                                                                                 │
│  2. 📸⚡ 拍照新宠降临！50 MP自拍镜头+Galaxy AI智能优化，每一帧都自带高光滤镜。Z                                 │
│  Flip7，不止是手机，更是你的移动时尚单品。#AI摄影新体验 #ZFlip7潮拍                                             │
│                                                                                                                 │
│  3. 🧠💡 Galaxy AI加持，让科技更懂你。从语音助手到健康监测，Z                                                   │
│  Flip7用智能与创新，为你开启折叠生活的无限可能。#智能新境界 #ZFlip7 AI                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CrewOutput(raw='1. 📱✨ 折叠未来，掌心绽放！Samsung Galaxy Z Flip7，以500,000次耐用铰链与轻薄设计，重新定义潮流手机。外屏AMOLED 120Hz刷新率，一瞥即惊艳。#ZFlip7 #折叠屏新纪元\n\n2. 📸⚡ 拍照新宠降临！50 MP自拍镜头+Galaxy AI智能优化，每一帧都自带高光滤镜。Z Flip7，不止是手机，更是你的移动时尚单品。#AI摄影新体验 #ZFlip7潮拍\n\n3. 🧠💡 Galaxy AI加持，让科技更懂你。从语音助手到健康监测，Z Flip7用智能与创新，为你开启折叠生活的无限可能。#智能新境界 #ZFlip7 AI', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='读取./results/datas.md内容，总结市场分析的关键数据点，生成数据摘要和竞品分析', name='读取./results/datas.md内容，总结市场分析的关键数据点，生成数据摘要和竞品分析', expected_output='一份包含核心数据和关键发现的数据摘要,一份竞品分析的报告', summary='读取./results/datas.md内容，总结市场分析的关键数据点，生成数据摘要和竞品分析...', raw='```\n**数据摘要**\n\n1. **产品概述**\n   - **发布日期**：2025年7月9日（正式发布），7月25日上市\n   - **市场定位**：紧凑型折叠屏手机，面向时尚潮流引领者和科技爱好者\n   - **核心功能**：\n     - 外部AMOLED显示屏（120Hz刷新率）\n     - 50 MP自拍相机\n     - 改进的铰链设计（500,000次折叠寿命）\n     - 更轻薄机身设计\n     - Galaxy AI功能增强\n\n2. **市场趋势**\n   - 折叠屏市场增长显著，2024年全球出货量达1720万台，同比增长12%\n   - 三星在折叠屏市场的份额从54%下降至45%\n   - 预计到2026年，折叠屏将占三星美国市场销量的15-20%\n\n3. **用户反馈**\n   - **正面评价**：外观设计时尚、外部屏幕便捷、